In [1]:
! pysradb metadata PRJNA1012098 | head

study_accession	study_title	experiment_accession	experiment_title	experiment_desc	organism_taxid	organism_name	library_name	library_strategy	library_source	library_selection	library_layout	sample_accession	sample_title	biosample	bioproject	instrument	instrument_model	instrument_model_desc	total_spots	total_size	run_accession	run_total_spots	run_total_bases
SRP458077	Transcriptional effects of AXL signaling and phosphorylation site mutants	SRX21599967	RNAseq of PC9: AXL signaling variants	RNAseq of PC9: AXL signaling variants	9606	Homo sapiens	M4EA	RNA-Seq	TRANSCRIPTOMIC	Oligo-dT	PAIRED	SRS18773628		SAMN37234672	PRJNA1012098	Illumina NovaSeq 6000	Illumina NovaSeq 6000	ILLUMINA	29004879	2823016364	SRR25879419	29004879	8701463700
SRP458077	Transcriptional effects of AXL signaling and phosphorylation site mutants	SRX21599966	RNAseq of PC9: AXL signaling variants	RNAseq of PC9: AXL signaling variants	9606	Homo sapiens	M4E	RNA-Seq	TRANSCRIPTOMIC	Oligo-dT	PAIRED	SRS18773626		SAMN37234671	PRJN

In [ ]:
path = "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations"

In [7]:
! pysradb download -y -t 8 --out-dir /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations -p PRJNA1012098

Checking download URLs
The following files will be downloaded: 

run_accession study_accession experiment_accession public_url                                                                                                  download_url                                                                                            out_dir                                                    filesize
SRR25879419   SRP458077       SRX21599967          https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos5/sra-pub-zq-14/SRR025/25879/SRR25879419/SRR25879419.lite.1 ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByRun/sra/SRR/SRR258/SRR25879419/SRR25879419.sra /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations 2.2 GB  
SRR25879420   SRP458077       SRX21599966          https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos5/sra-pub-zq-14/SRR025/25879/SRR25879420/SRR25879420.lite.1 ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByRun/sra/SRR/SRR258/SRR25879420/SRR25879420.sra /home/crei

In [4]:
! pwd

/home/creixell/AXLomics


In [5]:
! cd /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/FASTQ

In [ ]:
import os
import subprocess

base_dir = "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/SRP458077"

for root, dirs, files in os.walk(base_dir):
    for file in files:
        print(file)
        file_path = os.path.join(root, file)
        subprocess.run(["fasterq-dump", file_path])

AXL-Y726F-UT.sra


spots read      : 25,070,085
reads read      : 50,140,170
reads written   : 50,140,170


AXL-WT-UT.sra


spots read      : 29,337,615
reads read      : 58,675,230
reads written   : 58,675,230


AXL-Y821-UT.sra


...

AXL-KO-UT.sra


spots read      : 28,645,721
reads read      : 57,291,442
reads written   : 57,291,442


AXL-Y750F-UT.sra


spots read      : 24,376,856
reads read      : 48,753,712
reads written   : 48,753,712


AXL-Y634F-UT.sra


In [1]:
import os
import subprocess

def detect_egfr_erlotinib_resistance(
    fastq1, fastq2, reference_fasta, star_index, egfr_bed, gtf, out_prefix, threads=8
):
    """
    Aligns RNA-seq reads, extracts EGFR region, and calls EGFR variants (including exon 19 del) using STAR and GATK Mutect2.
    """
    # 1. Align reads to genome using STAR
    star_out = f"{out_prefix}_STAR"
    aligned_bam = star_out + "_Aligned.sortedByCoord.out.bam"
    if not os.path.exists(aligned_bam):
        subprocess.run([
            "STAR", "--runThreadN", str(threads),
            "--genomeDir", star_index,
            "--readFilesIn", fastq1, fastq2,
            "--readFilesCommand", "zcat" if fastq1.endswith(".gz") else "cat",
            "--outFileNamePrefix", star_out + "_",
            "--outSAMtype", "BAM", "SortedByCoordinate",
            "--twopassMode", "Basic",
            "--sjdbGTFfile", gtf
        ], check=True)
    else:
        print(f"Found existing BAM: {aligned_bam}, skipping STAR alignment.")

    # 2. Index the BAM file
    subprocess.run(["samtools", "index", aligned_bam], check=True)

    # 3. Extract only EGFR region (to speed up variant calling)
    egfr_bam = f"{out_prefix}.EGFR.bam"
    subprocess.run([
        "samtools", "view", "-b", "-L", egfr_bed, "-o", egfr_bam, aligned_bam
    ], check=True)
    subprocess.run(["samtools", "index", egfr_bam], check=True)

    # 3b. ***Add read group with sample name (required for GATK)***
    egfr_bam_rg = f"{out_prefix}.EGFR.rg.bam"
    sample_name = os.path.basename(out_prefix)   # Or set a string, e.g. "PAR-UT"
    subprocess.run([
        "samtools", "addreplacerg",
        "-r", f"@RG\\tID:1\\tSM:{sample_name}\\tLB:lib1\\tPL:ILLUMINA",
        "-o", egfr_bam_rg, egfr_bam
    ], check=True)
    subprocess.run(["samtools", "index", egfr_bam_rg], check=True)

    # 4. Call variants with GATK Mutect2 (NO '--rna' flag, just standard Mutect2 for RNA-bam)
    egfr_vcf = f"{out_prefix}.EGFR.mutect2.vcf"
    subprocess.run([
        "gatk", "Mutect2",
        "-R", reference_fasta,
        "-I", egfr_bam_rg,
        "-L", egfr_bed,
        "-O", egfr_vcf
    ], check=True)

    # 5. Filter Mutect2 calls
    filtered_vcf = f"{out_prefix}.EGFR.mutect2.filtered.vcf"
    subprocess.run([
        "gatk", "FilterMutectCalls",
        "-R", reference_fasta,
        "-V", egfr_vcf,
        "-O", filtered_vcf
    ], check=True)

    print(f"Mutect2 variant calling for EGFR complete. See filtered VCF: {filtered_vcf}")


In [3]:
detect_egfr_erlotinib_resistance(
    "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/AXL-PAR-UT_1.fastq", 
    "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/AXL-PAR-UT_2.fastq", 
    '/home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa', 
    '/home/creixell/ERAP/Allotyping/assets/star_index', 
    "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/egfr_exons.bed", 
    '/home/creixell/ERAP/Allotyping/assets/gencode.v44.annotation.gtf', 
    "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT",
    threads=8
)

Found existing BAM: /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT_STAR_Aligned.sortedByCoord.out.bam, skipping STAR alignment.


Using GATK jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar Mutect2 -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT.EGFR.rg.bam -L /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/egfr_exons.bed -O /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT.EGFR.mutect2.vcf
23:02:32.912 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
23:02:33.627 INFO  Mutect2 - -------------------------------------------

Tool returned:
SUCCESS


Using GATK jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar FilterMutectCalls -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa -V /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT.EGFR.mutect2.vcf -O /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT.EGFR.mutect2.filtered.vcf
23:02:46.957 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
23:02:47.207 INFO  FilterMutectCalls - ------------------------------------------------------------
23:02:47.212 INFO  Filter

Mutect2 variant calling for EGFR complete. See filtered VCF: /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Results/PAR-UT.EGFR.mutect2.filtered.vcf


In [14]:
! samtools addreplacerg \
    -r '@RG\tID:1\tSM:PAR-UT\tLB:lib1\tPL:ILLUMINA' \
    -o /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam \
    /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.bam


In [15]:
! samtools index /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam

In [18]:
!gatk HaplotypeCaller \
  -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa \
  -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam \
  -O /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.egfr.g.vcf.gz \
  -L chr7:55000000-55220000 \
  --dont-use-soft-clipped-bases true \
  --native-pair-hmm-threads 4 \
  --standard-min-confidence-threshold-for-calling 10

Using GATK jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar HaplotypeCaller -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam -O /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.egfr.g.vcf.gz -L chr7:55000000-55220000 --dont-use-soft-clipped-bases true --native-pair-hmm-threads 4 --standard-min-confidence-threshold-for-calling 10
11:28:35.416 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar!/com/intel/gkl/native/libgkl_compr

In [23]:
! gatk HaplotypeCaller \
  -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa \
  -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam \
  -O /tmp/all_variants.g.vcf.gz \
  --dont-use-soft-clipped-bases true \
  --native-pair-hmm-threads 4


Using GATK jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar HaplotypeCaller -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam -O /tmp/all_variants.g.vcf.gz --dont-use-soft-clipped-bases true --native-pair-hmm-threads 4
11:31:37.542 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
11:31:37.734 INFO  HaplotypeCaller - ------------------------------------------------------------
11:31:37.738 INFO  HaplotypeCaller - 

In [28]:
!gatk HaplotypeCaller \
  -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa \
  -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam \
  -O /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.egfr.del19.vcf.gz \
  -L chr7:55174739-55174819 \
  --dont-use-soft-clipped-bases false \
  --min-pruning 1 \
  --standard-min-confidence-threshold-for-calling 10 \
  --native-pair-hmm-threads 4

Using GATK jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar HaplotypeCaller -R /home/creixell/ERAP/Allotyping/assets/GRCh38.p14.genome.fa -I /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.EGFR.rg.bam -O /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.egfr.del19.vcf.gz -L chr7:55174739-55174819 --dont-use-soft-clipped-bases false --min-pruning 1 --standard-min-confidence-threshold-for-calling 10 --native-pair-hmm-threads 4
11:50:18.245 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/creixell/miniconda3/envs/pysradb/share/gatk4-4.6.2.0-0/gatk-package-4.6.2.0-local.jar!/com/intel/gk

In [32]:
! bcftools view -R "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/egfr_exons.3col.bed" "/home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.egfr.del19.vcf.gz"

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FILTER=<ID=LowQual,Description="Low quality">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
##GATKCommandLine=<ID=HaplotypeCaller,CommandLine="HaplotypeCaller --standard-min-confidence-threshold-for-calling 10.0 --min-pruning 1 --native-pair-hmm-threads 4 --dont-use-soft-clipped-bases false --output /home/creixell/AXLomics/msresist/data/RNAseq/EGFRmutations/Res_PC9-PAR-UT/PAR-UT.egfr.del19.vcf.gz --intervals chr7:55174739-55174819 --input /home/creixell